In [1]:
import numpy as np
import scipy as sp
import scipy.stats as stats

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch 
import torchvision


## Zadanie 1

Wytrenuj sieć na danych  z pliku `reg_data_noise.npy`. 

In [3]:
data = np.load('reg_multi.npy')

In [4]:
xs = data[:, :-1]
ys = data[:,-1]

In [31]:
t_xs = torch.from_numpy(xs) #.view(16,60000)
t_ys = torch.from_numpy(ys).view(-1,1)

In [32]:
model = torch.nn.Sequential(torch.nn.Linear(16,5),
                    torch.nn.ReLU(), torch.nn.Linear(5,1))

In [49]:
t_out = model(t_xs)

In [44]:
dataset = torch.utils.data.TensorDataset(t_xs, t_ys)
(train_set, valid_set) = torch.utils.data.random_split(dataset,(50000,10000))

In [45]:
train_loader = torch.utils.data.DataLoader(train_set,batch_size=len(train_set))

In [46]:
optimizer = torch.optim.SGD(model.parameters(),lr =0.001)

In [47]:
loss_func = torch.nn.MSELoss()

In [48]:
%%time
err_train=[]
err_valid=[]
for epoch in range(10000):    
    for datum in train_loader:
        optimizer.zero_grad()
        (features,target) = datum
        pred = model(features)
        loss = loss_func(pred, target)
        loss.backward()
        optimizer.step()
    
    with torch.no_grad():
        vpred  = model(valid_set[:][0])
        vloss  = loss_func(vpred,valid_set[:][1])
        err_valid.append(vloss)
        pred  = model(train_set[:][0])
        loss  = loss_func(pred,train_set[:][1])
        err_train.append(loss)
    if epoch%1000 == 0:
        print("epoch %d %f %f " % (epoch, loss, vloss))   

epoch 0 0.387321 0.388853 


KeyboardInterrupt: 

In [ ]:
plt.plot(err_train, '.', label='train')
plt.plot(err_valid, '.', label='valid');
plt.legend()

In [ ]:
t_out_train = model(train_set[:][0])
plt.plot(train_set[:][0].data.numpy(), t_out_train.data.view(-1).numpy(),'.')
t_out_valid = model(valid_set[:][0])
plt.plot(valid_set[:][0].data.numpy(), t_out_valid.data.view(-1).numpy(),'.')

In [ ]:
torch.save(model, "model.pt")